In [1]:
from src.AlphabetPermutation import AlphabetPermutation
from src.Probability import ProbabilityMatrix
from string import ascii_lowercase
import numpy as np

In [14]:
alphabet = list(ascii_lowercase)
alphabet.append(' ')

# Initialize an instance of permutation generator class
permutation_generator = AlphabetPermutation(alphabet)

# Obtain the key: in a cipher the key is the alphabet used by the cipher
key = permutation_generator.permutation.values()

In [15]:
message = "Hello MCMC let us see how well you decrypt messages"

In [16]:
permutation_generator.permute_string(message)

'dzwwi tbtb wze fo ozz dix xzww sif mzbhske tzoojqzo'